## 패키지 설치

In [ ]:
!pip install langchain langchain-core langchain-google-genai

## ChatGPT대신 Gemini를 사용하도록 수정한 first-langchain.py

힌트: <code>(github)class-2026-lginnotek-llm/first-langchain.py</code>를 Gemini를 이용해 수정

프롬프트:

> [다음 코드에서 OpenAI 를 사용하고 있는데 GoogleGenerativeAI 를 사용하도록 수정해줘]
>
> {first-langchain.py 코드 copy & paste}

In [ ]:
# 1. 라이브러리 임포트 및 API 키 설정
import os
# OpenAI 대신 GoogleGenerativeAI(ChatGoogleGenerativeAI)를 임포트합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Google API Key 설정 (환경변수 이름이 GOOGLE_API_KEY로 변경됩니다)
os.environ["GOOGLE_API_KEY"] = "your-api-key"

In [ ]:
# 2. 모델 초기화
# Gemini 모델을 사용하기 위해 ChatGoogleGenerativeAI를 초기화합니다.
# model 매개변수에 'gemini-1.5-flash' 또는 'gemini-pro' 등을 지정합니다.
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest",
    temperature=0.7
)

# 3. 프롬프트 템플릿 생성 (기존과 동일)
template = "{topic}에 대해 배울 수 있는 좋은 책 3권을 추천해줘."
prompt_template = PromptTemplate.from_template(template)

# 4. LCEL을 사용한 체인 구성 (기존과 동일)
# 프롬프트 -> LLM -> 문자열 출력 파서
chain = prompt_template | llm | StrOutputParser()

In [ ]:
# 5. 체인 실행 (기존과 동일)
response = chain.invoke({"topic": "인공지능"})
print(response)

## 최종 출력을 JSON형식으로 포메팅하여 출력하도록 수정

프롬프트:
> llm 의 출력을 포메팅해서 JSON 형식으로 만들고 싶어. 추천한 3권에 대해 키 "title", "author", "reason_of_recommendation", "key_features" 라고하고 각각, 제목, 저자, 추천이유, 특징을 값으로 넣어서 포메팅해줘.

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

In [ ]:
# 1. 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest",
    temperature=0.7
)

# 2. 프롬프트 템플릿 작성
# Pydantic 대신 프롬프트 안에 직접 JSON 예시를 보여줍니다.
# 주의: PromptTemplate 안에서 JSON의 중괄호 {}를 표현하려면 {{ }}로 두 번 감싸야 합니다.
template = """
{topic}에 대해 배울 수 있는 좋은 책 3권을 추천해줘.

반드시 아래와 같은 JSON 리스트 형식으로만 답변해줘 (다른 텍스트나 설명은 제외):

```json
[
  {{
    "title": "책 제목",
    "author": "저자",
    "reason_of_recommendation": "추천 이유",
    "key_features": "특징"
  }},
  ...
]
```
"""

prompt_template = PromptTemplate.from_template(template)

# 3. 파서 설정
# Pydantic 객체 없이 기본 파서만 사용합니다.
# 이 파서는 모델의 출력에서 JSON 부분만 추출하여 파이썬 딕셔너리/리스트로 변환해줍니다.
parser = JsonOutputParser()

# 4. 체인 구성
chain = prompt_template | llm | JsonOutputParser()

In [ ]:
# 5. 실행
response = chain.invoke({"topic": "인공지능"})

# 결과 확인 (이미 딕셔너리 리스트 형태로 변환되어 있음)
print(response)

In [ ]:
# 보기 좋게 출력하고 싶다면
import json
print(json.dumps(response, indent=2, ensure_ascii=False))